In [7]:
import math

In [6]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

# --- Configuration ---
API_KEY = " REMOVED_KEY"

# Define the area and grid size as before
AREA_BOUNDS = {
    "sw": (28.542092, 77.025868),  # South-West corner (lat, lon)
    "ne": (28.741011, 77.307083)  # North-East corner
}
GRID_SIZE_X = 5
GRID_SIZE_Y = 5
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0


for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

     
        # Request directions for this specific pair
        directions_result = gmaps.directions(origin,
                                                destination,
                                                mode="driving",
                                                departure_time=datetime.now())

        if not directions_result:
            continue

        # --- Extract Key Information ---
        route = directions_result[0] # Use the primary recommended route
        leg = route['legs'][0]
        
        # Calculate congestion factor for this specific path
        duration_val = leg['duration']['value']
        duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
        congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

        # --- Decode the Polyline to get actual road coordinates ---
        encoded_polyline = route['overview_polyline']['points']
        decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

        # Create a unique ID for this path
        path_id = f"Point {i} -> Point {j}"
        
        # For each coordinate in the decoded path, create a row for Tableau
        for order, point in enumerate(decoded_path):
            all_path_points.append({
                'Path_ID': path_id,
                'Point_Order': order + 1,
                'Latitude': point['lat'],
                'Longitude': point['lng'],
                'Congestion_Factor': round(congestion_factor, 2),
                'Distance_Text': leg['distance']['text'],
                'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
            })




# Convert the final list into a DataFrame
tableau_df = pd.DataFrame(all_path_points)

# Save the detailed path data to a new CSV file
tableau_df.to_csv(OUTPUT_FILENAME, index=False)

print(f"\n Successfully generated detailed path data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")

  Processing path 1/600 (Point 0 -> Point 1)
  Processing path 2/600 (Point 0 -> Point 2)
  Processing path 3/600 (Point 0 -> Point 3)
  Processing path 4/600 (Point 0 -> Point 4)
  Processing path 5/600 (Point 0 -> Point 5)
  Processing path 6/600 (Point 0 -> Point 6)
  Processing path 7/600 (Point 0 -> Point 7)
  Processing path 8/600 (Point 0 -> Point 8)
  Processing path 9/600 (Point 0 -> Point 9)
  Processing path 10/600 (Point 0 -> Point 10)
  Processing path 11/600 (Point 0 -> Point 11)
  Processing path 12/600 (Point 0 -> Point 12)
  Processing path 13/600 (Point 0 -> Point 13)
  Processing path 14/600 (Point 0 -> Point 14)
  Processing path 15/600 (Point 0 -> Point 15)
  Processing path 16/600 (Point 0 -> Point 16)
  Processing path 17/600 (Point 0 -> Point 17)
  Processing path 18/600 (Point 0 -> Point 18)
  Processing path 19/600 (Point 0 -> Point 19)
  Processing path 20/600 (Point 0 -> Point 20)
  Processing path 21/600 (Point 0 -> Point 21)
  Processing path 22/600 (Point

In [3]:
tableau_df

,Path_ID,Point_Order,Latitude,Longitude,Congestion_Factor,Distance_Text,Duration_in_Traffic_Text
0,Point 0 -> Point 1,1,28.48000,76.93017,1.00,3.0 km,9 mins
1,Point 0 -> Point 1,2,28.47953,76.93015,1.00,3.0 km,9 mins
2,Point 0 -> Point 1,3,28.47958,76.93033,1.00,3.0 km,9 mins
3,Point 0 -> Point 1,4,28.47974,76.93075,1.00,3.0 km,9 mins
4,Point 0 -> Point 1,5,28.47982,76.93090,1.00,3.0 km,9 mins
...,...,...,...,...,...,...,...
8848,Point 8 -> Point 7,49,28.52003,76.94858,1.11,3.2 km,9 mins
8849,Point 8 -> Point 7,50,28.51917,76.94813,1.11,3.2 km,9 mins
8850,Point 8 -> Point 7,51,28.51917,76.94864,1.11,3.2 km,9 mins
8851,Point 8 -> Point 7,52,28.51917,76.94953,1.11,3.2 km,9 mins


In [ ]:
import folium

location = [28.7041, 77.1025 ]
m = folium.Map(location=location, zoom_start=13)


# Google Maps Satellite View
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(m)

# Google Maps Traffic View
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=m@221097413,traffic&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Traffic',
    overlay=False,
    control=True
).add_to(m)

folium.LayerControl().add_to(m)

m